***
## Проектируем поздравления

Поздравления — это отдельная сущность, для них понадобится создать собственную модель; назовём её `Congratulation` (англ. «поздравление»).

* У поздравления должен быть текст — ради него эта фича и придумана.

* Поздравление должно быть связано с конкретной записью о дне рождения (с объектом модели `Birthday`) — значит, в модели `Congratulation` должен быть внешний ключ, ссылающийся на модель `Birthday`.

* У поздравления есть автор — аутентифицированный пользователь проекта; анонимные пользователи не смогут писать поздравления. В модели поздравлений понадобится внешний ключ со ссылкой на модель пользователя.

* Добавим в модель `Congratulation` поле, где будет сохраняться время создания поздравления; это поле должно заполняться автоматически, в момент создания нового объекта.

В шаблон *birthday_detail.html* добавим форму для отправки поздравлений и на эту же страницу выведем уже отправленные поздравления.

> Система комментирования когда-то была встроена в Django по умолчанию, но в [версии 1.6](https://docs.djangoproject.com/en/5.1/releases/1.6/#django-contrib-comments) её выделили в [отдельный проект](https://github.com/django/django-contrib-comments) и удалили из базовой установки.

***
## Модель поздравления

В файле *birthday/models.py* опишите модель `Congratulation`:


In [ ]:
# birthday/models.py
...


class Congratulation(models.Model):
    text = models.TextField('Текст поздравления')
    birthday = models.ForeignKey(
        Birthday, 
        on_delete=models.CASCADE,
        related_name='congratulations',
    )
    created_at = models.DateTimeField(auto_now_add=True)
    author = models.ForeignKey(User, on_delete=models.CASCADE)

    class Meta:
        ordering = ('created_at',) 

***
## Форма для поздравлений

Форма для отправки поздравления будет выводиться на странице записи о дне рождения; подготовим её.

Автор поздравления должен заполнить только поле text, а остальные поля должны заполняться автоматически; в HTML-форму должно быть выведено лишь одно поле.

Добавьте в файл *birthday/forms.py* класс формы, связанной с моделью Congratulation:


In [ ]:
# birthday/forms.py
...
# Импортируем модель поздравления.
from .models import Birthday, Congratulation
...


class CongratulationForm(forms.ModelForm):
    
    class Meta:
        model = Congratulation
        fields = ('text',) 

***
## Роутинг и view

Добавьте новый маршрут к *birthday/urls.py* 


In [ ]:
# birthday/urls.py
...
    path('<int:pk>/comment/', views.add_comment, name='add_comment'),
... 

На этот адрес будут отправляться POST-запросы на добавление поздравления.

Создайте view-функцию для обработки отправленного поздравления:


In [ ]:
# birthday/views.py
...
# Добавьте нужные импорты:
from django.contrib.auth.decorators import login_required
from django.shortcuts import get_object_or_404, redirect
...

from .forms import CongratulationForm
...


# Будут обработаны POST-запросы только от залогиненных пользователей.
@login_required
def add_comment(request, pk):
    # Получаем объект дня рождения или выбрасываем 404 ошибку.
    birthday = get_object_or_404(Birthday, pk=pk)
    # Функция должна обрабатывать только POST-запросы.
    form = CongratulationForm(request.POST)
    if form.is_valid():
        # Создаём объект поздравления, но не сохраняем его в БД.
        congratulation = form.save(commit=False)
        # В поле author передаём объект автора поздравления.
        congratulation.author = request.user
        # В поле birthday передаём объект дня рождения.
        congratulation.birthday = birthday
        # Сохраняем объект в БД.
        congratulation.save()
    # Перенаправляем пользователя назад, на страницу дня рождения.
    return redirect('birthday:detail', pk=pk) 

Теперь нужно доработать CBV `BirthdayDetailView`: передать в словарь контекста 

* форму для добавления поздравления,

* объекты поздравлений, связанные с запрошенным объектом `Birthday`.

Импортируйте нужные объекты в *birthday/views.py* и дополните код класса `BirthdayDetailView`:


In [ ]:
# birthday/views.py
...
from django.shortcuts import get_object_or_404, redirect

from .forms import BirthdayForm, CongratulationForm
from .models import Birthday, Congratulation

...

class BirthdayDetailView(DetailView):
    model = Birthday

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['birthday_countdown'] = calculate_birthday_countdown(
            self.object.birthday
        )
        # Записываем в переменную form пустой объект формы.
        context['form'] = CongratulationForm()
        # Запрашиваем все поздравления для выбранного дня рождения.
        context['congratulations'] = (
            # Дополнительно подгружаем авторов комментариев,
            # чтобы избежать множества запросов к БД.
            self.object.congratulations.select_related('author')
        )
        return context 

Если вместо CBV используется view-функция, то новые объекты добавляются в словарь контекста аналогичным способом, принципиальных отличий нет.

***
## Форма и поздравления в шаблоне страницы

Выводим в шаблон форму для добавления поздравлений и созданные поздравления:


In [ ]:

<!-- birthday_detail.html -->
{% extends "base.html" %}
<!-- Подгружаем теги для библиотеки django_bootstrap -->
{% load django_bootstrap5 %}

{% block content %}
  ID записи: {{ object.id }}
  <hr>
  {% if birthday.image %}
    <div>
      <img src="{{ birthday.image.url }}" height="200">
    </div>
  {% endif %}
  <h2>Привет, {{ object.first_name }} {{ object.last_name }}</h2>      
  {% if birthday_countdown == 0 %}
    <p>С днём рождения!</p>
  {% else %}
    <p>Осталось дней до дня рождения: {{ birthday_countdown }}!</p>
  {% endif %}
  <div>
    <h3>Поздравления:</h3>
    <!-- Если пользователь залогинен, отображаем форму -->
    {% if user.is_authenticated %}
      <!-- Указываем, на какой адрес нужно отправить POST-запрос -->
      <form action="{% url 'birthday:add_comment' object.pk %}" method="post" class="col-3">
        {% csrf_token %}
        {% bootstrap_form form %}
        {% bootstrap_button button_type="submit" content="Отправить поздравление" %}
      </form>
    {% endif %}
    <!-- В цикле перебираем и выводим поздравления -->
    {% for congratulation in congratulations %}
      <hr>
      <p>
        {{ congratulation.author.username }}
        поздравил {{ congratulation.created_at|date:"d M Y H:i" }}
      </p>
      <p><b>{{ congratulation.text }}</b></p>
    {% endfor %}
  </div>
{% endblock content %} 

***
## Обработать данные из формы CongratulationForm можно и с помощью CBV.

Класс `CongratulationCreateView` выполняет ту же работу, что и функция `add_comment()`.


In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin
from django.shortcuts import get_object_or_404
from django.urls import reverse
from django.views.generic import CreateView

from .forms import CongratulationForm
from .models import Birthday, Congratulation


class CongratulationCreateView(LoginRequiredMixin, CreateView):
    birthday = None
    model = Congratulation
    form_class = CongratulationForm

    # Переопределяем dispatch()
    def dispatch(self, request, *args, **kwargs):
        self.birthday = get_object_or_404(Birthday, pk=kwargs['pk'])
        return super().dispatch(request, *args, **kwargs)

    # Переопределяем form_valid()
    def form_valid(self, form):
        form.instance.author = self.request.user
        form.instance.birthday = self.birthday
        return super().form_valid(form)

    # Переопределяем get_success_url()
    def get_success_url(self):
        return reverse('birthday:detail', kwargs={'pk': self.birthday.pk}) 


Метод `dispatch()` выполняется ближе к началу обработки запроса (полный порядок обработки запроса есть [в документации](https://docs.djangoproject.com/en/5.1/ref/class-based-views/base/#view)); он проверяет, что в базе есть объект дня рождения с переданным в запросе `pk`. Если объект есть — он будет присвоен атрибуту `self.birthday`, если же объекта нет — будет выброшена ошибка 404. 

В методе `form_valid()` полям `author` и `birthday` нового объекта присваиваются нужные значения. Такую операцию вы уже описывали в CBV `BirthdayCreateView` — там автор присваивается новому объекту `Birthday`.

После отправки формы с поздравлением надо перенаправить пользователя на страницу записи. Это делается с помощью метода `get_success_url()`. В нём применяется функция `reverse()`, которая из имени маршрута и нужных аргументов собирает строку с адресом; это аналог тега `{% url "birthday:detail" pk %}`, но только в Python-коде.

А чтобы вся эта красота заработала — надо поменять маршрут и вместо функции `add_comment()` указать `CongratulationCreateView.as_view()`, как и полагается при работе с CBV.